In [1]:
import numpy as np

from scipy.sparse import load_npz

In [2]:
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn import metrics

In [3]:
from imblearn.over_sampling import RandomOverSampler

### Reading data

In [4]:
Xtrain = load_npz('features_silico_duplicated.npz')
Xtrain

<151627x1676 sparse matrix of type '<class 'numpy.float64'>'
	with 1901453 stored elements in Compressed Sparse Row format>

In [5]:
Ytrain = np.load('classes_silico_duplicated.npy')
Ytrain.shape

(151627, 71)

In [8]:
def predict(clf, params, X, Y, n_splits=5):
    accuracytrain = []
    accuracytest = []
    hammingtrain = []
    hammingtest = []
    f1train = []
    f1test = []
    precisiontrain = []
    precisiontest = []
    recalltrain = []
    recalltest = []

    kf = KFold(n_splits=n_splits)
    for train_idx, test_idx in kf.split(X):
        Xtrain = X[train_idx]
        Xtest = X[test_idx]
        Ytrain = Y[train_idx]
        Ytest = Y[test_idx]

        Ytrainpred_arr = []
        Ytestpred_arr = []

        for i in range(71):
            
            ytrain = Ytrain[:,i]
            ytest = Ytest[:,i]
            
            if np.unique(ytrain).size == 1:
                ytrainpred = np.full(ytrain.shape, ytrain[0])
                ytestpred = np.full(ytest.shape, ytrain[0])
            else:
                ros = RandomOverSampler()
                Xtrain_resampled, ytrain_resampled = ros.fit_sample(Xtrain, ytrain)
                
                gscv = GridSearchCV(clf, params, scoring='accuracy', cv=5, n_jobs=-1)
                gscv.fit(Xtrain_resampled, ytrain_resampled)
                print(gscv.best_score_)
                print(gscv.best_params_)
            
            ytrainpred = gscv.predict(Xtrain)
            ytestpred = gscv.predict(Xtest)
            Ytrainpred_arr.append(ytrainpred)
            Ytestpred_arr.append(ytestpred)

        Ytrainpred = np.array(Ytrainpred_arr).T
        Ytestpred = np.array(Ytestpred_arr).T

        Ytrain = (Ytrain==1).astype('int')
        Ytest = (Ytest==1).astype('int')
        Ytrainpred = (Ytrainpred==1).astype('int')
        Ytestpred = (Ytestpred==1).astype('int')

        accuracytrain.append(metrics.accuracy_score(Ytrain, Ytrainpred))
        accuracytest.append(metrics.accuracy_score(Ytest, Ytestpred))
        hammingtrain.append(1 - metrics.hamming_loss(Ytrain, Ytrainpred))
        hammingtest.append(1 - metrics.hamming_loss(Ytest, Ytestpred))
        f1train.append(metrics.f1_score(Ytrain, Ytrainpred, average='micro'))
        f1test.append(metrics.f1_score(Ytest, Ytestpred, average='micro'))
        precisiontrain.append(metrics.precision_score(Ytrain, Ytrainpred, average='micro'))
        precisiontest.append(metrics.precision_score(Ytest, Ytestpred, average='micro'))
        recalltrain.append(metrics.recall_score(Ytrain, Ytrainpred, average='micro'))
        recalltest.append(metrics.recall_score(Ytest, Ytestpred, average='micro'))

    print('Accuracy: \t \t {} \t {}'.format(np.array(accuracytrain).mean(), np.array(accuracytest).mean()))
    print('Hamming: \t \t {} \t {}'.format(np.array(hammingtrain).mean(), np.array(hammingtest).mean()))
    print('Precision: \t \t {} \t {}'.format(np.array(precisiontrain).mean(), np.array(precisiontest).mean()))
    print('Recall: \t \t {} \t {}'.format(np.array(recalltrain).mean(), np.array(recalltest).mean()))
    print('F1: \t \t \t {} \t {}'.format(np.array(f1train).mean(), np.array(f1test).mean()))

In [9]:
params = {'C': [1.0, 10.0, 100.0]}

In [10]:
predict(LogisticRegression(penalty='l2', solver='sag', max_iter=1000, n_jobs=-1), params, Xtrain, Ytrain)

/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/uufs/chpc.utah.edu/

0.8725704336815447
{'C': 1.0}


/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/uufs/chpc.utah.edu/

0.9023355319992276
{'C': 1.0}


/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/uufs/chpc.utah.edu/

...

0.942834690431206
{'C': 1.0}


/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/uufs/chpc.utah.edu/

0.938906523505222
{'C': 1.0}


/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/uufs/chpc.utah.edu/

0.9182808410031242
{'C': 1.0}
Accuracy: 	 	 0.06658945027154081 	 0.0556502411633647
Hamming: 	 	 0.9166980731955698 	 0.9099438218957998
Precision: 	 	 0.3398169813737109 	 0.31747229538179733
Recall: 	 	 0.8955976998786033 	 0.8780245623619727
F1: 	 	 	 0.49221850322207716 	 0.46371046734242005


In [11]:
predict(LinearSVC(penalty='l2'), params, Xtrain, Ytrain)

0.8758388730610953
{'C': 10.0}
0.9138505788013375
{'C': 1.0}
0.875770503295745
{'C': 1.0}
0.9510079357840008
{'C': 100.0}
0.9523633491842118
{'C': 100.0}
0.9670416809049409
{'C': 1.0}
0.9755735927068153
{'C': 10.0}
0.9999402771141902
{'C': 1.0}
0.9958986720726399
{'C': 1.0}
0.9948365622599642
{'C': 100.0}
0.9645901254640998
{'C': 1.0}
0.9688373866621472
{'C': 1.0}
0.9288989862509763
{'C': 1.0}
0.9895583095562281
{'C': 1.0}
0.9774758351704257
{'C': 1.0}
0.8990228751501961
{'C': 100.0}
0.9364902813255968
{'C': 1.0}
0.9996715185229426
{'C': 100.0}
0.9795149367603676
{'C': 10.0}
0.9864165311885
{'C': 1.0}
0.9920487148455022
{'C': 1.0}
0.9833948339483395
{'C': 100.0}
0.9682116093082584
{'C': 1.0}
0.9790235142358505
{'C': 1.0}
0.9365143533136452
{'C': 1.0}
0.9226997330047237
{'C': 100.0}
0.932093480067491
{'C': 10.0}
0.9974156726406438
{'C': 1.0}
0.9466234161690322
{'C': 10.0}
0.9159364268435031
{'C': 100.0}
0.9016052544822865
{'C': 100.0}
0.9638168711268497
{'C': 1.0}
0.9427291810077421
{'C